Last time I explored how to index a data set I created exploring three different methodologies for creating stats for 5E D&D characters.  In this blog post, I'm going to run some descriptive statistics as well as some exploratory data analysis.

In [1]:
#import functions
import pandas as pd
import matplotlib.pyplot as plt

#for in notebook graphic exploration
%matplotlib inline

In [2]:
#read in the data
df = pd.read_csv('1000CharSimulated20seed.csv',index_col=0)

In [3]:
#use the shape function to look at the data
df.shape

#shape gives us the number of rows and the number of columns
#unlike head() or tail(), shape is an attribute of the data frame, not a method 

(3000, 13)

In [4]:
#use describe to get some high level statistics
df.describe()

,strength,dexterity,constitution,intellegence,wisdom,charisma,str mod,dex mod,con mod,int mod,wis mod,char mod
count,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,11.915333,11.990333,12.007333,12.015000,11.947000,11.866333,0.701667,0.737667,0.751667,0.754333,0.730000,0.676000
std,3.074191,3.187417,3.112851,3.126665,3.150794,3.142784,1.547728,1.616494,1.575275,1.582872,1.593303,1.589927
min,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,-4.000000,-4.000000,-4.000000,-4.000000,-4.000000,-4.000000
25%,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,14.000000,15.000000,14.000000,15.000000,14.000000,14.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
max,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000


The describe function gives us a lot of great information on each of the variables including:
* Count
* Mean
* Standard Deviation
* Whisker PLot Inputs (min, 25th percentile 50th percentile, 75th percentile, and max)

Notice how describe doesn't provide any summary statistics for the qualitative roll_type variable.

While this is useful, it is not quite fair to summarize all the data together because it represents the aggregation of three different methodologies for creating characters.  So what I really need is a way to run describe() three different times, one for each methodology. Luckily, I can do that with groupby

In [5]:
df.groupby('roll_type').describe()
#looks way better on Mac, but not sure why

char mod     charisma      con mod  constitution  \
roll_type                                                               
3D6        count  1000.000000  1000.000000  1000.000000   1000.000000   
           mean     -0.088000    10.361000    -0.033000     10.444000   
           std       1.540334     3.028506     1.494036      2.937600   
           min      -4.000000     3.000000    -4.000000      3.000000   
           25%      -1.000000     8.000000    -1.000000      8.000000   
           50%       0.000000    10.000000     0.000000     11.000000   
           75%       1.000000    13.000000     1.000000     13.000000   
           max       4.000000    18.000000     4.000000     18.000000   
4D6DropLow count  1000.000000  1000.000000  1000.000000   1000.000000   
           mean      0.825000    12.139000     0.990000     12.455000   
           std       1.448611     2.861095     1.430366      2.818496   
           min      -4.000000     3.000000    -3.000000      4.000000   
           25%       0.000000    10.000000     0.000000     11.000000   
           50%       1.000000    12.000000     1.000000     13.000000   
           75%       2.000000    14.000000     2.000000     15.000000   
           max       4.000000    18.000000     4.000000     18.000000   
Colville   count  1000.000000  1000.000000  1000.000000   1000.000000   
           mean      1.291000    13.099000     1.298000     13.123000   
           std       1.460292     2.903382     1.483036      2.936408   
           min      -3.000000     5.000000    -3.000000      4.000000   
           25%       0.000000    11.000000     0.000000     11.000000   
           50%       1.000000    13.000000     2.000000     14.000000   
           75%       2.000000    15.000000     2.000000     15.000000   
           max       4.000000    18.000000     4.000000     18.000000   

                      dex mod    dexterity      int mod  intellegence  \
roll_type                                                               
3D6        count  1000.000000  1000.000000  1000.000000   1000.000000   
           mean     -0.032000    10.433000     0.021000     10.554000   
           std       1.545109     3.044138     1.507591      2.956658   
           min      -4.000000     3.000000    -4.000000      3.000000   
           25%      -1.000000     8.000000    -1.000000      9.000000   
           50%       0.000000    10.000000     0.000000     11.000000   
           75%       1.000000    13.000000     1.000000     13.000000   
           max       4.000000    18.000000     4.000000     18.000000   
4D6DropLow count  1000.000000  1000.000000  1000.000000   1000.000000   
           mean      0.887000    12.292000     1.012000     12.528000   
           std       1.476622     2.878371     1.438724      2.835711   
           min      -3.000000     4.000000    -4.000000      3.000000   
           25%       0.000000    10.000000     0.000000     11.000000   
           50%       1.000000    12.000000     1.000000     13.000000   
           75%       2.000000    14.000000     2.000000     15.000000   
           max       4.000000    18.000000     4.000000     18.000000   
Colville   count  1000.000000  1000.000000  1000.000000   1000.000000   
           mean      1.358000    13.246000     1.230000     12.963000   
           std       1.508679     2.974951     1.532138      3.043829   
           min      -3.000000     4.000000    -3.000000      4.000000   
           25%       0.000000    11.000000     0.000000     11.000000   
           50%       2.000000    14.000000     1.000000     13.000000   
           75%       3.000000    16.000000     2.000000     15.000000   
           max       4.000000    18.000000     4.000000     18.000000   

                      str mod     strength      wis mod       wisdom  
roll_type                                                             
3D6        count  1000.000000  1000.000000  1000.000000  1000.000000  
          

That's way better.  We can also use groupby to look at specific statistics.  For example, let's look at the average for each statistic by the roll_type.

In [7]:
df.groupby('roll_type').mean()

,strength,dexterity,constitution,intellegence,wisdom,charisma,str mod,dex mod,con mod,int mod,wis mod,char mod
roll_type,,,,,,,,,,,,
3D6,10.374,10.433,10.444,10.554,10.608,10.361,-0.069,-0.032,-0.033,0.021,0.062,-0.088
4D6DropLow,12.319,12.292,12.455,12.528,12.089,12.139,0.909,0.887,0.990,1.012,0.807,0.825
Colville,13.053,13.246,13.123,12.963,13.144,13.099,1.265,1.358,1.298,1.230,1.321,1.291


In [8]:
df.groupby('roll_type').median()

,strength,dexterity,constitution,intellegence,wisdom,charisma,str mod,dex mod,con mod,int mod,wis mod,char mod
roll_type,,,,,,,,,,,,
3D6,10,10,11,11,11,10,0,0,0,0,0,0
4D6DropLow,12,12,13,13,12,12,1,1,1,1,1,1
Colville,13,14,14,13,14,13,1,2,2,1,2,1


Looking above we can see that Colville Method is higher for each average and greater than or equal to each median of the statistics and for each of the modifiers.  They are also not that different from one to the other (which makes sense.  They were generated the same way, so they should look roughly the same).  

So let's take it a step further and add in a column to calculate the average ability score and the average modifier for each row

In [14]:
df['ability_mean'] = df.loc[:,"strength":"charisma"].mean(axis=1)
df['mod_mean'] = df.loc[:,"str mod":"char mod"].mean(axis=1)

In [25]:
df.groupby('roll_type)'[["ability_mean","mod_mean"]]

SyntaxError: unexpected EOF while parsing (<ipython-input-25-12ccee667ec1>, line 1)